# Chapter 35: TypeScript with Angular

Angular is unique among major frontend frameworks as it was built with TypeScript from the ground up. Unlike React or Vue where TypeScript is optional, Angular treats TypeScript as a first-class citizen, leveraging decorators, type checking, and interfaces to provide a comprehensive development experience. This chapter covers Angular's architecture, strict typing patterns, and advanced TypeScript features specific to the Angular ecosystem.

---

## 35.1 Angular's Built-in TypeScript Architecture

Angular's core philosophy embraces TypeScript's type system, decorators, and module resolution to enforce architectural patterns at compile time.

### Project Setup with Strict Mode

```bash
# Create new Angular project with strict mode (recommended)
ng new my-angular-app --strict --standalone --routing --style=scss

# Or with legacy module-based architecture
ng new my-angular-app --strict --routing --style=scss

cd my-angular-app
ng serve
```

**tsconfig.json (Strict Configuration):**
```json
{
  "compileOnSave": false,
  "compilerOptions": {
    "baseUrl": "./",
    "outDir": "./dist/out-tsc",
    "forceConsistentCasingInFileNames": true,
    "strict": true,
    "noImplicitOverride": true,
    "noPropertyAccessFromIndexSignature": true,
    "noImplicitReturns": true,
    "noFallthroughCasesInSwitch": true,
    "sourceMap": true,
    "declaration": false,
    "downlevelIteration": true,
    "experimentalDecorators": true,
    "emitDecoratorMetadata": true,
    "moduleResolution": "node",
    "importHelpers": true,
    "target": "ES2022",
    "module": "ES2022",
    "useDefineForClassFields": false,
    "lib": ["ES2022", "dom"],
    "paths": {
      "@app/*": ["src/app/*"],
      "@env/*": ["src/environments/*"],
      "@shared/*": ["src/app/shared/*"]
    }
  },
  "angularCompilerOptions": {
    "enableI18nLegacyMessageIdFormat": false,
    "strictInjectionParameters": true,
    "strictInputAccessModifiers": true,
    "strictTemplates": true,
    "strictNullChecks": true,
    "strictFunctionTypes": true,
    "strictPropertyInitialization": true,
    "noImplicitAny": true,
    "noImplicitReturns": true,
    "noImplicitThis": true,
    "strictDomEventTypes": true,
    "strictSafeNavigationTypes": true,
    "strictContextGenerics": true,
    "strictLiteralTypes": true
  }
}
```

**Explanation:**
- `strict: true`: Enables all TypeScript strict mode checks
- `angularCompilerOptions`: Angular-specific type checking for templates
- `strictTemplates`: Validates types in component templates (bindings, pipes, etc.)
- `strictInjectionParameters`: Ensures injectable services have proper types
- `paths`: Configured for clean imports using TypeScript path mapping

### Standalone Components Architecture (Angular 15+)

```typescript
// main.ts - Bootstrap with standalone components
import { bootstrapApplication } from '@angular/platform-browser';
import { provideRouter } from '@angular/router';
import { provideHttpClient } from '@angular/common/http';
import { importProvidersFrom } from '@angular/core';
import { AppComponent } from './app/app.component';
import { routes } from './app/app.routes';

bootstrapApplication(AppComponent, {
  providers: [
    provideRouter(routes),
    provideHttpClient(),
    // Import ngModule-based providers if needed
    importProvidersFrom(SomeLegacyModule)
  ]
}).catch(err => console.error(err));

// app.component.ts - Standalone root component
import { Component } from '@angular/core';
import { RouterOutlet } from '@angular/router';
import { HeaderComponent } from './shared/components/header/header.component';

@Component({
  selector: 'app-root',
  standalone: true,
  imports: [RouterOutlet, HeaderComponent], // Import dependencies directly
  template: `
    <app-header />
    <main>
      <router-outlet />
    </main>
  `,
  styles: []
})
export class AppComponent {
  title = 'my-angular-app';
}
```

---

## 35.2 Typing Components with Decorators

Angular uses decorators extensively to define metadata while TypeScript handles type definitions.

### Component Decorator Types

```typescript
import { Component, Input, Output, EventEmitter, OnInit, OnDestroy } from '@angular/core';
import { CommonModule } from '@angular/common';

// Interface for component data
interface User {
  id: string;
  name: string;
  email: string;
  role: 'admin' | 'user' | 'guest';
  isActive: boolean;
  metadata?: Record<string, unknown>;
}

@Component({
  selector: 'app-user-card',
  standalone: true,
  imports: [CommonModule],
  template: `
    <div class="user-card" [class.active]="user.isActive">
      <h3>{{ user.name }}</h3>
      <p>{{ user.email }}</p>
      <span class="badge" [ngClass]="'role-' + user.role">
        {{ user.role | uppercase }}
      </span>
      <button (click)="onSelect()">View Details</button>
    </div>
  `,
  styles: [`
    .user-card { border: 1px solid #ccc; padding: 16px; }
    .active { border-color: green; }
  `]
})
export class UserCardComponent implements OnInit, OnDestroy {
  // Typed Input with required option (Angular 16+)
  @Input({ required: true }) user!: User;
  
  // Optional Input with default value
  @Input() showBadge: boolean = true;
  
  // Input with transform function
  @Input({ transform: (value: string | number) => Number(value) }) 
  priority: number = 0;

  // Typed Output events
  @Output() select = new EventEmitter<User>();
  @Output() delete = new EventEmitter<string>();

  // Private state with explicit typing
  private subscription: Subscription | null = null;
  isLoading: boolean = false;

  ngOnInit(): void {
    // Lifecycle hook implementation
    this.initializeComponent();
  }

  ngOnDestroy(): void {
    // Cleanup
    this.subscription?.unsubscribe();
  }

  private initializeComponent(): void {
    // Component initialization logic
  }

  onSelect(): void {
    // Type-safe event emission
    this.select.emit(this.user);
  }

  onDelete(): void {
    this.delete.emit(this.user.id);
  }
}
```

**Explanation:**
- `implements OnInit, OnDestroy`: Enforces correct lifecycle method signatures
- `@Input({ required: true })`: TypeScript enforces that required inputs are provided
- `!`: Non-null assertion tells TypeScript the input will be provided
- `@Output()`: EventEmitter is generic, emitting typed data
- Template type checking validates `user.name`, `user.isActive` exist on User type

### Typed Template References

```typescript
@Component({
  selector: 'app-search',
  template: `
    <input #searchInput type="text" (keyup.enter)="search(searchInput.value)" />
    <button (click)="search(searchInput.value)">Search</button>
    
    <form #form="ngForm" (ngSubmit)="submitForm(form)">
      <input name="email" ngModel required email />
    </form>
  `
})
export class SearchComponent {
  @ViewChild('searchInput') searchInput!: ElementRef<HTMLInputElement>;
  @ViewChild('form') form!: NgForm;

  search(query: string): void {
    // query is typed as string
    console.log(this.searchInput.nativeElement.value);
  }

  submitForm(form: NgForm): void {
    if (form.valid) {
      console.log(form.value);
    }
  }
}
```

---

## 35.3 Dependency Injection and Service Typing

Angular's DI system is fully typed, ensuring services are correctly provided and injected.

### Injectable Services

```typescript
// models/user.model.ts
export interface User {
  id: string;
  name: string;
  email: string;
  createdAt: Date;
}

export interface CreateUserDTO {
  name: string;
  email: string;
  password: string;
}

// services/user.service.ts
import { Injectable, inject } from '@angular/core';
import { HttpClient } from '@angular/common/http';
import { Observable, BehaviorSubject, throwError } from 'rxjs';
import { catchError, map, shareReplay } from 'rxjs/operators';
import { User, CreateUserDTO } from '../models/user.model';

// Injection token for configuration
export const API_URL = new InjectionToken<string>('api.url');

interface ApiResponse<T> {
  data: T;
  success: boolean;
  message?: string;
}

@Injectable({
  providedIn: 'root' // Tree-shakable provider
})
export class UserService {
  // Modern inject() function (Angular 14+)
  private http = inject(HttpClient);
  private apiUrl = inject(API_URL);
  
  // State management with typed subjects
  private users$ = new BehaviorSubject<User[]>([]);
  private loading$ = new BehaviorSubject<boolean>(false);
  private error$ = new BehaviorSubject<string | null>(null);

  // Exposed observables with explicit types
  readonly users: Observable<User[]> = this.users$.asObservable();
  readonly loading: Observable<boolean> = this.loading$.asObservable();
  readonly error: Observable<string | null> = this.error$.asObservable();

  getUsers(): Observable<User[]> {
    this.loading$.next(true);
    
    return this.http.get<ApiResponse<User[]>>(`${this.apiUrl}/users`)
      .pipe(
        map(response => response.data),
        catchError(this.handleError),
        shareReplay(1)
      );
  }

  getUserById(id: string): Observable<User> {
    return this.http.get<ApiResponse<User>>(`${this.apiUrl}/users/${id}`)
      .pipe(
        map(response => response.data),
        catchError(this.handleError)
      );
  }

  createUser(userData: CreateUserDTO): Observable<User> {
    return this.http.post<ApiResponse<User>>(`${this.apiUrl}/users`, userData)
      .pipe(
        map(response => response.data),
        catchError(this.handleError)
      );
  }

  updateUser(id: string, updates: Partial<User>): Observable<User> {
    return this.http.patch<ApiResponse<User>>(`${this.apiUrl}/users/${id}`, updates)
      .pipe(
        map(response => response.data),
        catchError(this.handleError)
      );
  }

  deleteUser(id: string): Observable<void> {
    return this.http.delete<void>(`${this.apiUrl}/users/${id}`)
      .pipe(catchError(this.handleError));
  }

  private handleError(error: HttpErrorResponse): Observable<never> {
    let errorMessage = 'An unknown error occurred';
    
    if (error.error instanceof ErrorEvent) {
      // Client-side error
      errorMessage = `Error: ${error.error.message}`;
    } else {
      // Server-side error
      errorMessage = `Error Code: ${error.status}\nMessage: ${error.message}`;
    }
    
    this.error$.next(errorMessage);
    return throwError(() => new Error(errorMessage));
  }
}
```

**Explanation:**
- `inject()` function provides type-safe dependency injection without constructor boilerplate
- `InjectionToken<T>` creates typed injection tokens for configuration values
- Generic `Observable<T>` ensures operators maintain type safety through the chain
- Private subjects with public observables encapsulate state while maintaining type safety

### Service with Local Storage

```typescript
import { Injectable } from '@angular/core';

interface StorageConfig {
  prefix: string;
  ttl?: number;
}

@Injectable()
export class LocalStorageService {
  constructor(private config: StorageConfig) {}

  setItem<T>(key: string, value: T): void {
    const item = {
      value,
      timestamp: Date.now(),
      ttl: this.config.ttl
    };
    localStorage.setItem(`${this.config.prefix}:${key}`, JSON.stringify(item));
  }

  getItem<T>(key: string): T | null {
    const data = localStorage.getItem(`${this.config.prefix}:${key}`);
    if (!data) return null;

    try {
      const item = JSON.parse(data) as { value: T; timestamp: number; ttl?: number };
      
      if (item.ttl && Date.now() - item.timestamp > item.ttl) {
        this.removeItem(key);
        return null;
      }
      
      return item.value;
    } catch {
      return null;
    }
  }

  removeItem(key: string): void {
    localStorage.removeItem(`${this.config.prefix}:${key}`);
  }

  clear(): void {
    Object.keys(localStorage)
      .filter(key => key.startsWith(this.config.prefix))
      .forEach(key => localStorage.removeItem(key));
  }
}

// Factory provider with configuration
export function provideLocalStorage(config: StorageConfig) {
  return {
    provide: LocalStorageService,
    useFactory: () => new LocalStorageService(config)
  };
}
```

---

## 35.4 RxJS Integration with TypeScript

RxJS is integral to Angular. TypeScript ensures operators maintain type safety through transformations.

### Typed Observables and Operators

```typescript
import { Component, OnInit } from '@angular/core';
import { FormControl } from '@angular/forms';
import { Observable, Subject, combineLatest, of } from 'rxjs';
import { 
  map, 
  filter, 
  switchMap, 
  catchError, 
  debounceTime, 
  distinctUntilChanged,
  takeUntil,
  tap 
} from 'rxjs/operators';

interface Product {
  id: number;
  name: string;
  price: number;
  category: string;
}

interface CartItem extends Product {
  quantity: number;
}

@Component({
  selector: 'app-product-search',
  template: `
    <input [formControl]="searchControl" placeholder="Search products..." />
    <div *ngIf="loading">Loading...</div>
    <ul>
      <li *ngFor="let product of products$ | async">
        {{ product.name }} - {{ product.price | currency }}
      </li>
    </ul>
  `
})
export class ProductSearchComponent implements OnInit, OnDestroy {
  searchControl = new FormControl<string>('');
  loading = false;
  
  private destroy$ = new Subject<void>();
  
  // Typed observable chain
  products$!: Observable<Product[]>;

  constructor(private productService: ProductService) {}

  ngOnInit(): void {
    this.products$ = this.searchControl.valueChanges.pipe(
      // Type inference maintains string type
      debounceTime(300),
      distinctUntilChanged(),
      filter((term): term is string => term !== null && term.length > 2),
      tap(() => this.loading = true),
      switchMap((term: string) => 
        this.productService.search(term).pipe(
          catchError(error => {
            console.error(error);
            return of([]); // Return typed empty array
          })
        )
      ),
      tap(() => this.loading = false),
      takeUntil(this.destroy$)
    );
  }

  ngOnDestroy(): void {
    this.destroy$.next();
    this.destroy$.complete();
  }
}

// Service with complex operators
@Injectable()
export class ProductService {
  private products$ = new BehaviorSubject<Product[]>([]);
  
  // Combine multiple streams with strict typing
  getFilteredProducts(
    category$: Observable<string>,
    priceRange$: Observable<{ min: number; max: number }>
  ): Observable<Product[]> {
    return combineLatest([this.products$, category$, priceRange$]).pipe(
      map(([products, category, range]) => 
        products.filter(product => 
          product.category === category &&
          product.price >= range.min &&
          product.price <= range.max
        )
      )
    );
  }

  // Generic caching operator
  cache<T>(key: string): MonoTypeOperatorFunction<T> {
    return (source: Observable<T>) => {
      if (this.cacheMap.has(key)) {
        return this.cacheMap.get(key) as Observable<T>;
      }
      
      const cached$ = source.pipe(shareReplay(1));
      this.cacheMap.set(key, cached$);
      return cached$;
    };
  }
  
  private cacheMap = new Map<string, Observable<unknown>>();
}
```

---

## 35.5 Forms: Template-driven and Reactive

Angular offers two form approaches, both fully typable, with Reactive Forms providing better type safety.

### Typed Reactive Forms

```typescript
import { Component } from '@angular/core';
import { 
  FormBuilder, 
  FormGroup, 
  FormControl, 
  Validators, 
  AbstractControl,
  ValidationErrors 
} from '@angular/forms';

// Strictly typed form model
interface ProfileForm {
  firstName: FormControl<string>;
  lastName: FormControl<string>;
  email: FormControl<string>;
  address: FormGroup<{
    street: FormControl<string>;
    city: FormControl<string>;
    zip: FormControl<string>;
  }>;
  preferences: FormGroup<{
    newsletter: FormControl<boolean>;
    notifications: FormControl<'email' | 'sms' | 'push'>;
  }>;
}

@Component({
  selector: 'app-profile',
  template: `
    <form [formGroup]="profileForm" (ngSubmit)="onSubmit()">
      <input formControlName="firstName" placeholder="First Name" />
      <div *ngIf="profileForm.get('firstName')?.invalid && profileForm.get('firstName')?.touched">
        First name is required
      </div>
      
      <div formGroupName="address">
        <input formControlName="street" placeholder="Street" />
        <input formControlName="city" placeholder="City" />
        <input formControlName="zip" placeholder="ZIP" />
      </div>
      
      <button type="submit" [disabled]="profileForm.invalid">Save</button>
    </form>
  `
})
export class ProfileComponent {
  profileForm: FormGroup<ProfileForm>;

  constructor(private fb: FormBuilder) {
    this.profileForm = this.fb.group({
      firstName: ['', [Validators.required, Validators.minLength(2)]],
      lastName: ['', Validators.required],
      email: ['', [Validators.required, Validators.email]],
      address: this.fb.group({
        street: ['', Validators.required],
        city: ['', Validators.required],
        zip: ['', [Validators.required, Validators.pattern(/^\d{5}$/)]]
      }),
      preferences: this.fb.group({
        newsletter: [false],
        notifications: ['email' as const]
      })
    });
  }

  onSubmit(): void {
    if (this.profileForm.valid) {
      // TypeScript knows the exact shape of value
      const formValue = this.profileForm.value;
      
      // formValue.firstName is string | undefined
      // formValue.address?.city is string | undefined
      
      this.saveProfile(formValue as ProfileFormValue);
    }
  }

  private saveProfile(data: ProfileFormValue): void {
    // API call with typed data
  }
}

// Helper type to get value type from form
type ProfileFormValue = ReturnType<FormGroup<ProfileForm>['getRawValue']>;
```

### Custom Validators with Types

```typescript
// Validator function type
export function forbiddenNameValidator(nameRe: RegExp): ValidatorFn {
  return (control: AbstractControl): ValidationErrors | null => {
    const forbidden = nameRe.test(control.value);
    return forbidden ? { forbiddenName: { value: control.value } } : null;
  };
}

// Async validator
export function uniqueEmailValidator(userService: UserService): AsyncValidatorFn {
  return (control: AbstractControl): Observable<ValidationErrors | null> => {
    return userService.checkEmailExists(control.value).pipe(
      map(isTaken => (isTaken ? { uniqueEmail: true } : null)),
      catchError(() => of(null))
    );
  };
}

// Usage in component
this.profileForm = this.fb.group({
  username: ['', {
    validators: [Validators.required, forbiddenNameValidator(/admin/i)],
    asyncValidators: [uniqueEmailValidator(this.userService)],
    updateOn: 'blur'
  }]
});
```

---

## 35.6 HTTP Client with Typed Requests

Angular's HttpClient uses generics to maintain type safety across network boundaries.

### Typed HTTP Operations

```typescript
import { Injectable } from '@angular/core';
import { HttpClient, HttpParams, HttpHeaders, HttpErrorResponse } from '@angular/common/http';
import { Observable, throwError } from 'rxjs';
import { catchError, map } from 'rxjs/operators';

// API Response wrapper
interface ApiResponse<T> {
  data: T;
  meta?: {
    total: number;
    page: number;
    perPage: number;
  };
}

// Request/Response types
interface User {
  id: string;
  name: string;
  email: string;
}

interface CreateUserRequest {
  name: string;
  email: string;
  role: 'admin' | 'user';
}

interface UpdateUserRequest {
  name?: string;
  email?: string;
  role?: 'admin' | 'user';
}

@Injectable()
export class ApiService {
  private readonly baseUrl = '/api/v1';

  constructor(private http: HttpClient) {}

  // GET with typed response
  getUsers(page: number = 1, limit: number = 10): Observable<ApiResponse<User[]>> {
    const params = new HttpParams()
      .set('page', page.toString())
      .set('limit', limit.toString());

    return this.http.get<ApiResponse<User[]>>(`${this.baseUrl}/users`, { params })
      .pipe(catchError(this.handleError));
  }

  // GET single item
  getUserById(id: string): Observable<User> {
    return this.http.get<ApiResponse<User>>(`${this.baseUrl}/users/${id}`)
      .pipe(
        map(response => response.data),
        catchError(this.handleError)
      );
  }

  // POST with typed request and response
  createUser(userData: CreateUserRequest): Observable<User> {
    return this.http.post<ApiResponse<User>>(`${this.baseUrl}/users`, userData)
      .pipe(
        map(response => response.data),
        catchError(this.handleError)
      );
  }

  // PATCH with partial update
  updateUser(id: string, updates: UpdateUserRequest): Observable<User> {
    return this.http.patch<ApiResponse<User>>(`${this.baseUrl}/users/${id}`, updates)
      .pipe(
        map(response => response.data),
        catchError(this.handleError)
      );
  }

  // DELETE
  deleteUser(id: string): Observable<void> {
    return this.http.delete<void>(`${this.baseUrl}/users/${id}`)
      .pipe(catchError(this.handleError));
  }

  // Download with blob
  downloadReport(userId: string): Observable<Blob> {
    return this.http.get(`${this.baseUrl}/users/${userId}/report`, {
      responseType: 'blob'
    }).pipe(catchError(this.handleError));
  }

  // Upload with progress tracking
  uploadFile(file: File): Observable<HttpEvent<ApiResponse<{ url: string }>>> {
    const formData = new FormData();
    formData.append('file', file);

    return this.http.post<ApiResponse<{ url: string }>>(
      `${this.baseUrl}/upload`, 
      formData,
      {
        reportProgress: true,
        observe: 'events'
      }
    ).pipe(catchError(this.handleError));
  }

  private handleError(error: HttpErrorResponse): Observable<never> {
    let errorMessage = '';
    
    if (error.error instanceof ErrorEvent) {
      errorMessage = `Client Error: ${error.error.message}`;
    } else {
      errorMessage = `Server Error: ${error.status} - ${error.message}`;
    }
    
    return throwError(() => new Error(errorMessage));
  }
}
```

---

## 35.7 Guards, Interceptors, and Resolvers

Angular's routing system supports type-safe guards and interceptors.

### Route Guards

```typescript
import { Injectable } from '@angular/core';
import { 
  CanActivate, 
  CanActivateChild, 
  CanDeactivate, 
  Resolve,
  Router, 
  UrlTree 
} from '@angular/router';
import { Observable, of } from 'rxjs';
import { map, catchError } from 'rxjs/operators';
import { AuthService } from './auth.service';

// Component with canDeactivate interface
export interface CanComponentDeactivate {
  canDeactivate: () => Observable<boolean> | Promise<boolean> | boolean;
}

// Auth Guard
@Injectable({
  providedIn: 'root'
})
export class AuthGuard implements CanActivate, CanActivateChild {
  constructor(private authService: AuthService, private router: Router) {}

  canActivate():
    | Observable<boolean | UrlTree>
    | Promise<boolean | UrlTree>
    | boolean
    | UrlTree {
    return this.authService.isAuthenticated$.pipe(
      map(isAuth => {
        if (isAuth) return true;
        return this.router.createUrlTree(['/login']);
      })
    );
  }

  canActivateChild():
    | Observable<boolean | UrlTree>
    | Promise<boolean | UrlTree>
    | boolean
    | UrlTree {
    return this.canActivate();
  }
}

// Role-based Guard
@Injectable()
export class RoleGuard implements CanActivate {
  constructor(private authService: AuthService, private router: Router) {}

  canActivate(route: ActivatedRouteSnapshot): boolean | UrlTree {
    const requiredRoles = route.data['roles'] as string[];
    const userRole = this.authService.getUserRole();
    
    if (requiredRoles.includes(userRole)) {
      return true;
    }
    
    return this.router.createUrlTree(['/unauthorized']);
  }
}

// Unsaved Changes Guard
@Injectable()
export class UnsavedChangesGuard implements CanDeactivate<CanComponentDeactivate> {
  canDeactivate(
    component: CanComponentDeactivate
  ): Observable<boolean> | Promise<boolean> | boolean {
    return component.canDeactivate ? component.canDeactivate() : true;
  }
}

// Data Resolver
@Injectable()
export class UserResolver implements Resolve<User> {
  constructor(private userService: UserService) {}

  resolve(route: ActivatedRouteSnapshot): Observable<User> {
    const id = route.paramMap.get('id');
    if (!id) throw new Error('No user ID provided');
    
    return this.userService.getUserById(id).pipe(
      catchError(error => {
        // Navigate to error page or handle appropriately
        throw error;
      })
    );
  }
}

// Route configuration
export const routes: Routes = [
  {
    path: 'admin',
    canActivate: [AuthGuard, RoleGuard],
    data: { roles: ['admin'] },
    children: [
      {
        path: 'users/:id/edit',
        component: UserEditComponent,
        canDeactivate: [UnsavedChangesGuard],
        resolve: { user: UserResolver }
      }
    ]
  }
];
```

### HTTP Interceptors

```typescript
import { HttpInterceptorFn, HttpErrorResponse } from '@angular/common/http';
import { inject } from '@angular/core';
import { Router } from '@angular/router';
import { catchError, throwError } from 'rxjs';

// Functional interceptor (Angular 15+)
export const authInterceptor: HttpInterceptorFn = (req, next) => {
  const authService = inject(AuthService);
  const router = inject(Router);
  
  const token = authService.getToken();
  
  if (token) {
    req = req.clone({
      setHeaders: {
        Authorization: `Bearer ${token}`
      }
    });
  }
  
  return next(req).pipe(
    catchError((error: HttpErrorResponse) => {
      if (error.status === 401) {
        authService.logout();
        router.navigate(['/login']);
      }
      return throwError(() => error);
    })
  );
};

// Error handling interceptor
export const errorInterceptor: HttpInterceptorFn = (req, next) => {
  const notificationService = inject(NotificationService);
  
  return next(req).pipe(
    catchError((error: HttpErrorResponse) => {
      let message = 'An unknown error occurred';
      
      if (error.error instanceof ErrorEvent) {
        message = error.error.message;
      } else {
        switch (error.status) {
          case 400:
            message = error.error.message || 'Bad Request';
            break;
          case 403:
            message = 'Forbidden';
            break;
          case 404:
            message = 'Not Found';
            break;
          case 500:
            message = 'Internal Server Error';
            break;
        }
      }
      
      notificationService.showError(message);
      return throwError(() => error);
    })
  );
};

// Provider configuration
export const appConfig: ApplicationConfig = {
  providers: [
    provideRouter(routes),
    provideHttpClient(
      withInterceptors([authInterceptor, errorInterceptor])
    )
  ]
};
```

---

## 35.8 Chapter Summary and Exercises

### Chapter Summary

This chapter covered TypeScript in Angular development:

**Key Concepts:**

1. **Strict Mode**: Angular's strict TypeScript configuration enforces type safety in templates, dependency injection, and component interactions. `angularCompilerOptions` extends type checking to templates.

2. **Standalone Components**: Modern Angular (15+) uses standalone components with direct imports, eliminating NgModule boilerplate while maintaining full type safety through `imports` arrays.

3. **Decorators**: `@Input()`, `@Output()`, `@Injectable()` use TypeScript decorators with generic type parameters. Required inputs and transform functions (Angular 16+) enhance type safety.

4. **Dependency Injection**: The `inject()` function provides type-safe service injection. `InjectionToken<T>` creates typed configuration providers. Tree-shakable providers (`providedIn: 'root'`) maintain types across lazy-loaded boundaries.

5. **RxJS**: Observables maintain type safety through operator chains. Custom operators can be generic. `takeUntil` pattern with typed subjects ensures proper cleanup.

6. **Forms**: Reactive Forms with `FormGroup<Interface>` provide compile-time checking of form controls. Custom validators use `ValidatorFn` and `ValidationErrors` types.

7. **HTTP Client**: `HttpClient` methods are generic (`get<T>`, `post<T>`), ensuring request/response type consistency. Interceptors handle typed `HttpErrorResponse`.

8. **Routing**: Guards implement typed interfaces (`CanActivate`, `Resolve`). Route data can be typed using generics or discriminated unions.

### Practical Exercises

**Exercise 1: Type-Safe Component Library**

Build a reusable Data Grid component:
- Generic `@Input() data: T[]` with row type parameter
- Column definitions with `keyof T` for field access
- Sorting and filtering with typed operators
- Row selection emitting `T` or `T['id']`
- Template type checking for cell renderers

**Exercise 2: State Management Service**

Create a generic CRUD service:
- Abstract class `BaseService<T extends { id: string }>`
- Methods: `getAll()`, `getById(id: string)`, `create(dto: Omit<T, 'id'>)`, `update(id: string, changes: Partial<T>)`, `delete(id: string)`
- State management with `BehaviorSubject<T[]>`
- Optimistic updates with rollback on error
- Type-safe caching mechanism

**Exercise 3: Advanced Forms**

Implement a dynamic form builder:
- Configuration interface with field types (text, number, select, date)
- Form array support for repeatable sections
- Cross-field validation (e.g., password confirmation)
- Conditional field visibility based on other field values
- Strict typing ensuring form values match interface

**Exercise 4: Real-time Application**

Build a chat application with RxJS:
- WebSocket service with typed message protocols
- Message interface with discriminated union for types (text, image, system)
- Typing indicators using `Subject` and `debounceTime`
- Unread message count with `scan` operator
- Message history pagination with `BehaviorSubject`

**Exercise 5: Enterprise Authentication**

Create an auth system with:
- JWT handling interceptor with automatic refresh
- Role-based access control with typed route data
- Permission directives (`*appPermission="'admin'"`)
- Auth guard that redirects to original URL after login
- User profile resolver with caching

**Exercise 6: Testing Utilities**

Write typed test utilities:
- Mock services with partial implementations
- Component harnesses for consistent testing
- HTTP testing controller with typed expectations
- RxJS marble testing for complex operator chains

### Additional Resources

- **Angular TypeScript Configuration**: https://angular.io/guide/typescript-configuration
- **Strict Template Type Checking**: https://angular.io/guide/template-typecheck
- **Angular Dependency Injection**: https://angular.io/guide/dependency-injection
- **RxJS with Angular**: https://angular.io/guide/rx-library
- **Angular Architecture Patterns**: https://angular.io/guide/lazy-loading-ngmodules

---

## Coming Up Next: Chapter 36 - Testing TypeScript Code

In the next chapter, we will explore testing strategies for TypeScript applications:

- Unit testing with Jest and Vitest
- TypeScript-specific testing utilities
- Mocking strategies for typed dependencies
- Testing React components with React Testing Library
- Testing Vue components with Vue Test Utils
- Testing Node.js APIs with Supertest
- Integration testing strategies
- Property-based testing with fast-check
- Type testing with tsd

Understanding how to test TypeScript code ensures type safety is maintained throughout the development lifecycle, catching both logical errors and type mismatches in your test suites.